## Metabolomics:

### In order to analyse and report the analysis, I use jupyter notebook, along with Pandas to parse the given raw data file for further analysis. In order to use pandas, it is imported into the notebook first. 

### Parsing the Raw data file:

- Importing the modules:

In [1]:
# Importing the modules
import pandas as pd

- Reading the Excel file into a pandas's dataframe:

In [2]:
# Adding the path for the file location and it could be changed by any user to make use of it.
PATH = "C:/Users/mayur/Documents/GitHub/Intern_project/FileFolder/"

# Reading the Excel file as a dataframe using pandas and seting ID as a index row.
CC_metabolite = pd.read_excel(PATH + "cellC_meta.xlsx",index_col = "ID")

- Parsing the dataframe to remove the blank treatment rows to avoid redundancy:

In [3]:
# This command is to remove rows from a dataframe.
# Selecting the treatment column name with Blank rows and dropping them

filteredD = CC_metabolite.drop(CC_metabolite[CC_metabolite["Treatments"]=="Blank"].index)

- Splitting and changing the time column names to 30 and 90 mins:

In [4]:
# this splits the time column elements and stores as a list. we only need 30 and 90 mins from the given string.

str_split = filteredD["Time"].str.split("_",n=2,expand=True)[1]
list_str = list(str_split)

In [5]:
# Inserting the column to specific index position '2' as the new column is always added at the end of the dataframe table.
# Specifying to allow the duplicate names as 30 and 90 mins are...

filteredD.insert(2,"time", list_str,allow_duplicates=True)

In [6]:
# Dropping the original time column..

final_data = filteredD.drop(["Time"],axis=1)

- Mean values are calculated for duplicate values of Treatment and Time columns: 

In [7]:
# Using groupby for treatment and time columns as they have duplicated values and then calculating the mean values.
# Final data contains 16 rows of treatments and 70 columns of metabolite IDs.

avg_duplicated = final_data.groupby(["Treatments","time"]).mean().reset_index()

### Storing the Parsed file:

In [8]:
# saving the parsed file in the home directory in Excel format.

PATH = "C:/Users/mayur/Documents/GitHub/Intern_project/FileFolder/"

avg_duplicated.to_excel(PATH + "metabolite_data.xlsx")

### Subset the file into two based on TIME variable:

- For Subset 30 Mins:

In [9]:
# In this, the final data after parsing is taken and subset into 30 mins based on time column.
# Index is set to treatment as there are no duplicate names.

time_30min = avg_duplicated[avg_duplicated["time"] == "30Min"].set_index("Treatments")

In [10]:
# saving the subset data file in Excel format to use it in R.

time_30min.to_excel(PATH + "subset_30min.xlsx")

- For Subset 90 Mins:

In [11]:
# In this, the final data after parsing is taken and subset into 90 mins based on time column.
# Index is set to treatment as there are no duplicate names.

time_90min = avg_duplicated[avg_duplicated["time"] == "90Min"].set_index("Treatments")

In [12]:
# saving the subset data file in Excel format to use it in R.

time_90min.to_excel(PATH + "subset_90min.xlsx")

### Calculation of Fold Change Values for subset data:

In [13]:
#Writing a class to calculate the fold change values for both subset data
class Metabolomics:
    
    #Constructor takes in the subset data file each time
    def __init__(self,file):
        self.file = file
       
    #Writing a method parsing file
    def parsingfile(self):
        
        #taking first seven columns and saving it as treatment groups
        self.treatGrp = self.file.iloc[:,0:7]
        #taking last column from the file and saving it in a variable control group.
        self.controlGrp = self.file.iloc[:,7]
    
    #This method calculates the fold change values between each treatment group and control group
    def foldchange_cal(self):
        
        #assigning metabolite ids into list to a variable.
        index_lst = self.treatGrp.index.values.tolist()
       
        #assigning the local variables in form of list to make a dataframe file.
        treatments = []
        lst_values = []

        #iterating every treatment group column
        for i,j in self.treatGrp.iteritems():
            #calculating fold change values using ratio between treatment/control group
            foldchange = j/self.controlGrp
            #appending each of the values to list
            lst_values.append(foldchange.tolist())
            #appending every treatment group into a list
            treatments.append(i)
            
        #creating dataframe by taking in all the list variables into it.
        #treatment group as index and columns as metabolite ids.
        #finally transposing the dataframe and return it into an argument in main code.
        FC_data = pd.DataFrame(lst_values,index=treatments,columns=index_lst).T
        return FC_data

In [14]:
#dropping the column time from both subset data as it isn't required for this task and then transpose it.
time_30 = time_30min.drop("time",axis=1).T
time_90 = time_90min.drop("time",axis=1).T

################ subset 30 min ###################
#passing the 30 min subset data into a class metabolomics..
subset1 = Metabolomics(time_30)
#CALLING THE METHODS FROM THE GIVEN CLASS
subset1.parsingfile()
FC_data = subset1.foldchange_cal()
#returned dataframe fold change values is saved into an excel in particular path.
FC_data.to_excel(PATH + "FC_30min.xlsx")

################ subset 90 min ###################
#passing the 90 min subset data into a class metabolomics..
subset2 = Metabolomics(time_90)
#CALLING THE METHODS FROM THE GIVEN CLASS
subset2.parsingfile()
FC_data = subset2.foldchange_cal()
#returned dataframe fold change values is saved into an excel in particular path.
FC_data.to_excel(PATH + "FC_90min.xlsx")